In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from keras.models import Sequential, load_model, Model
from keras.layers import *
from keras import optimizers
from keras import utils
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import plot_model
from keras.models import save_model, load_model
import pydot

import keras

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time



In [ ]:
import telegram
from keras.callbacks import Callback
from callbacks import TelegramCallback
from callbacks.TelegramData import TelegramData


# create callback
config = {
    'token': TelegramData.Token,   # paste your bot token
    'telegram_id': TelegramData.ID,                                   # paste your telegram_id
    'model_name': 'JAN_LSTM_CNN_CROSS'
}

tg_callback = TelegramCallback(config)

In [ ]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

        tensor =  tf.convert_to_tensor(self.settings_str)
        summary = tf.summary.text ("Run_Settings", tensor)

        with  tf.Session() as sess:
            s = sess.run(summary)
            self.writer.add_summary(s)

In [ ]:
train_ids = [1,2,3, 7, 8, 9, 10,12,13,14,15,16]
test_ids = [4,5,6,11,17]

In [ ]:
dfAll = pd.read_pickle("PklData/df_lstm_norm50.pkl")
dfCNN =  pd.read_pickle("PklData/df_blobs.pkl")

In [ ]:
def f(row):
    if row['TaskID'] < 17:
        #val = "Knuckle"
        val = 0
    elif row['TaskID'] >= 17:
        #val = "Finger"
        val = 1
    return val
dfAll['InputMethod'] = dfAll.apply(f, axis=1)
dfAll.TaskID = dfAll.TaskID % 17

In [ ]:
dfAll.head()

In [ ]:
def createDataSets(train_ids, test_ids):
    df_train = dfAll[dfAll.userID.isin(train_ids)]
    df_test = dfAll[dfAll.userID.isin(test_ids)]

    df_train2 = df_train[['Blobs', 'TaskID', 'InputMethod']].copy()
    df_test2 = df_test[['Blobs', 'TaskID', 'InputMethod']].copy()
    x_train = np.concatenate(df_train2.Blobs.values).reshape(-1,50,27,15,1)
    x_test = np.concatenate(df_test2.Blobs.values).reshape(-1,50,27,15,1)

    y_train = df_train2.TaskID.values
    y_test = df_test2.TaskID.values

    y_train_cnn = df_train2.InputMethod.values
    y_test_cnn = df_train2.InputMethod.values

    num_classes = len(dfAll.TaskID.unique()) 
    y_train_one_hot = utils.to_categorical(df_train2.TaskID, num_classes)
    y_test_one_hot = utils.to_categorical(df_test2.TaskID, num_classes)
    
    # convert class vectors to binary class matrices (one-hot notation)
    num_classes_cnn = 2
    y_train_one_hot_cnn = utils.to_categorical(df_train2.InputMethod, num_classes_cnn)
    y_test_one_hot_cnn = utils.to_categorical(df_test2.InputMethod, num_classes_cnn)

In [ ]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
#config = tf.ConfigProto(device_count = {"GPU": 1})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.allow_growth = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.3

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [ ]:
KnuckelFinger = load_model("10_01_19.h5", compile=True)
KnuckelFinger.summary()



In [ ]:
batch_size = 60
epochs = 100
timesteps = 50
data_dim = (27,15)


def gesture_model():

    tf.get_default_graph()
    model = Sequential()

    #### Gesture structure
    A0 = Input(shape=(50,27,15,1),name='A0')
    A1 = TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', 
                                kernel_regularizer=regularizers.l2(0.01)),input_shape=(50, 27,15,1), name='A1')(A0)
    A2 = TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),name='A2')(A1)
    #model.add(TimeDistributed(BatchNormalization(axis=-1)))
    A3 = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last'),name='A3')(A2)
    A4 = TimeDistributed(Dropout(0.50), name='A4')(A3)


    #A5 = TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),name='A5')(A4)
    #A6 = TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),name='A6')(A5)
    #model.add(TimeDistributed(BatchNormalization(axis=-1)))
    #A7 = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last'),name='A7')(A6)
    #A8 = TimeDistributed(Dropout(0.50), name='A8')(A7)

    A9 = TimeDistributed(Flatten(), name='A9')(A4)

    A10 = LSTM(256, return_sequences=True, input_shape=(timesteps, data_dim), name='A10', kernel_regularizer=regularizers.l2(0.01))(A9)
    A11 = LSTM(128, kernel_regularizer=regularizers.l2(0.01), name='A11')(A10)
    A12 = Dense(num_classes, activation='softmax', name='A12')(A11)
    A = Model(inputs=A0, outputs=A12)
    # Output is a (17,1) tensor which tells us about finger or knuckle input

    ### END Gesture structure

    #print(KnuckelFinger.outputs[0])
    model = Model(inputs = KnuckelFinger.layers[0].input, outputs = KnuckelFinger.layers[-1].output)
    B0 = TimeDistributed(model, name='B0')(A0)
    B1 = Flatten(name="B1")(B0)
    B2 = Dense(2, activation='softmax', name='B2')(B1)
    #B1 = Reshape((27,15,-1),name='B1')(A0)

    #B2 = Conv2D(64 ,kernel_size=(3,3), input_shape=(30,27,15,1),activation='relu',name='B2')(B1)
    #B3 = Flatten(name='B3')(B2)
    #B3 = Dense(128, activation='relu', kernel_regularizer=keras.regularizers.L1L2(0.02, 0.15),name='B3', use_bias=True)(B2)
    #B3 = Dense(2, activation='softmax',name='B4')(B0)
    # Output is a (2,1) tensor which tells us about finger or knuckle input
    merged = Model(inputs=[A0],outputs=[A12, B2])
    config = ""
    for layer in merged.layers:
        config += str(layer.output).split('\"')[1].split("/")[0] + str(layer.output_shape) + "\n\n"
    #### END TENSORBOARD
    config += "batchsize: " + str(batch_size) + "\n\n" + "epochs: " + str(epochs) + "\n\n"

    #merged.add_weight(KnuckelFinger, shape=(27,15))
    optimizer = optimizers.Adam(lr = 0.0001, decay=1e-6)
    #optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
    merged.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return merged

In [ ]:


readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
tensorflowfolder = "/srv/share/tensorboardfiles/KnuckleFinger_Jan_LSTM+CNN" + readable_timestamp
logger = LoggingTensorBoard(settings_str_to_log = config, log_dir=tensorflowfolder, histogram_freq=0, write_graph=True, write_images=True, update_freq = 'epoch')
model_checkpoint = ModelCheckpoint("./ModelSnapshots/KnuckleFinger_Jan_LSTM+CNN" + readable_timestamp + ".h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=10, 
                                            verbose=1, 
                                            factor=0.95, 
                                            min_lr=0.00001)
storer = ModelCheckpoint("./ModelSnapshots/KnuckleFinger_Jan_LSTM+CNN" + readable_timestamp + ".h5", monitor='val_loss', verbose=0,
                         save_best_only=True, save_weights_only=False, mode='auto', period=1)



def fit_and_evaluate(x_train, y_train_one_hot, x_test, x_test_one_hot, EPOCHS=500, BATCH_SIZE=500):
    model = None
    model = gesture_model()
    model.summary()
    history = merged.fit(x_train, [y_train_one_hot, y_train_one_hot_cnn],
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, [y_test_one_hot, y_test_one_hot_cnn]),
                        callbacks=[storer, 
                                   logger,
                                   tg_callback])
    
    #print("Val Score: ", model.evaluate(val_x, val_y))
    return history

In [ ]:
n_folds = dfAll.userID.unique()
epochs=500
batch_size=500

#save the model history in a list after fitting so that we can plot later
model_history = []

readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
result_file = open("CrossValid/results_gestures_%s.csv" % readable_timestamp, "a")
result_file.write("TrainIds;TestIds;val_A12_acc;val_B2_acc;A12_acc;B2_acc\n")
result_file.flush()

for i in n_folds:
    test_ids = [i]
    
    train_ids = np.setdiff1d(dfAll.userID.unique(), test_ids)
     
    print("Training on Fold: ",i, "\n test_ids: ", test_ids, "\n train_ids: ", train_ids)
    x_train, y_train_one_hot, x_test, y_test_one_hot = createDataSets(train_ids, test_ids)
    #t_x, val_x, t_y, val_y = train_test_split(x_train, y_train, test_size=0.1, random_state = np.random.randint(1,1000, 1)[0])
    ####END
    cur_hist = fit_and_evaluate(x_train, y_train_one_hot, x_test, y_test_one_hot, epochs, batch_size)
    model_history.append(cur_hist)
    
    hist_result = str(train_ids) + ";" + str(test_ids) + ";"
    hist_result += str(cur_hist.history["val_A12_acc"]) + ";"
    hist_result += str(cur_hist.history["val_B2_acc"]) + ";"
    hist_result += str(cur_hist.history["A12_acc"]) + ";"
    hist_result += str(cur_hist.history["B2_acc"]) + "\n"
    
    result_file.write(hist_result)
    result_file.flush()
    
    print("======="*12, end="\n\n\n")

result_file.close()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['val_A12_acc'], label="Test Accuracy LSTM")
plt.plot(history.history['val_B2_acc'], label="Test Accuracy CNN")
plt.plot(history.history['A12_acc'], label="Training Accuracy LSTM")
plt.plot(history.history['B2_acc'], label="Training Accuracy CNN")

plt.legend()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    merged.save('lstm_cnn_27_01_19.h5')

In [ ]:
y_test_pred = merged.predict(x_test)

In [ ]:
y_test_pred[0] = np.argmax(y_test_pred[0], axis=1)

In [ ]:
y_test_pred[1] = np.argmax(y_test_pred[1], axis=1)

In [ ]:


print ('\n Summary of the precision, recall, F1 score for each class:')
print (sklearn.metrics.classification_report(y_test, y_test_pred[0]))

print ('\n Confusion matrix: ')
print (sklearn.metrics.confusion_matrix(y_test, y_test_pred[0]))

In [ ]:


print ('\n Summary of the precision, recall, F1 score for each class:')
print (sklearn.metrics.classification_report(y_test, y_test_pred[1]))

print ('\n Confusion matrix: ')
print (sklearn.metrics.confusion_matrix(y_test, y_test_pred[1]))